---
### ✅ **ANÁLISIS DEL SECTOR DE INTERNET EN ARGENTINA**
#### *La industria de las telecomunicaciones ha desempeñado un papel crucial en nuestra sociedad, facilitando la información a escala global y permitiendo la comunicación continua. La transferencia de datos y la comunicación se realizan principalmente a través de internet, líneas telefónicas fijas y móviles. Argentina está a la vanguardia en el desarrollo de las telecomunicaciones, contando con un total de 62,12 millones de conexiones en 2020. Dada la relevancia del tema para el país, he llevado a cabo un análisis exhaustivo que permite identificar el comportamiento de este sector a nivel nacional, enfocándome en el acceso al servicio de Internet y su relación con otros servicios de comunicaciones. El objetivo es generar recomendaciones para ofrecer una buena calidad de servicio, identificar oportunidades de crecimiento y plantear soluciones personalizadas para clientes actuales o potenciales.*
---

### 💡 **ETL del Proyecto**
<div style="text-align: justify;">
#### *A partir de fuentes de información como el portal de ENACOM (https://indicadores.enacom.gob.ar/datos-abiertos), se obtienen datos asociados al comportamiento histórico trimestral desde el año 2014 hasta el tercer trimestre de 2024 a nivel nacional y, en algunos casos, a nivel provincial. Todos estos datos están concentrados en múltiples hojas dentro de un solo archivo de Excel.*
</div>
### ✅ **Carga de archivos Crudos:**

1. **Se inicializa el proceso con la importacion del archivo "Internet.xlsx" el cual tiene la estructura de diccionario por contener varias hojas con multiples campos cada una.**
2. **Para facilitar el reconocimiento del contenido de cada campo en los dos dataframe macro se ajustan los nombres de los campos por nombres practicos.**
3. **En una primera exploración de las hojas se idetifica hay datos desagregados por año y trimestre a nivel nacional y hay otros con año, trimestre y provincia por lo cual podríamos reducir la cantidad de dataframes agrupando gran parte de la información en dos macro dataframes (df_Internet_Nacional, df_Internet_Provincias).**
4. **Aplicamos reduccionalidad al dataset luego de identificar la presencia de campos redundantes como "Trimestre.1" y campos comunes (Provincia, año, trimestre), creamos un campo id_año_trim_nal y id_año_trim_prov para todas las hojas donde aplique respectivamente, para cada hoja validamos dimensionalidad e identificamos si se generaria o no perdida de información evaluando el impacto que generaría unificar las hojas con los nuevos campos id.**
5. **Una vez se han unificado dataframes dejando los campos en comun con una sola ocurrencia, se generan copias de respaldo para seguir operando el ETL con estas.**
6. **Una vez revisado el tipo de información contenida en cada campo de los dataset unificados y simplificados, se crean diccionarios con tipos de datos para aplicar un casting y homogenizar la información previendo y mitigando problemas asociados a formato ya que vienen mas validaciones y analisis que requieren haber depurado estos aspectos**
7. ****

#2. **Con la funcion "split_xls_x_df" creamos un dataframe por cada hoja contenida en el archivo de excel.**


In [ ]:
#1. **Se inicializa el proceso con la importacion del archivo "Internet.xlsx" el cual tiene
# la estructura de diccionario por contener varias hojas con multiples campos cada una.**
import pandas as pd
from functions import opciones_impresion, renombrar_campos, opciones_impresion, obtener_hojas_validas, validar_df

# Invocar la función para mejorar la impresion:
opciones_impresion()

# Carga las hojas sin cargar datos, solo los nombres
df_internet = pd.read_excel('/Users/usuario/Documents/M7_LABs_PI/mvp_pi2/data_csv/raw/Internet.xlsx', sheet_name=None)
excel_data = pd.ExcelFile('/Users/usuario/Documents/M7_LABs_PI/mvp_pi2/data_csv/raw/Internet.xlsx')

# Ajustar Pandas para que no corte la impresión en varias filas
pd.set_option('display.expand_frame_repr', False)  # Muestra la tabla en una sola fila si la pantalla es ancha
pd.set_option('display.max_columns', None)         # Asegura que se muestren todas las columnas sin truncarlas
pd.set_option('display.width', 1000)               # Ajusta el ancho máximo permitido para la salida

# Validamos las primeras lineas de cada hoja para identifica su contenido
for hojas in df_internet:
    print(f'La hoja {hojas} contiene:') # Imprime el nombre de la hoja
    print(df_internet[hojas].head(3))
    print('\n')

#2. **Para facilitar el reconocimiento del contenido de cada campo en los dos dataframe macro se 
# **ajustan los nombres de los campos por nombres practicos.**

# Lista de nombres actuales y sus renombramientos para los campos de nivel nacional:
nuevos_nombres_nivel_nacional = {"Totales VMD":{   "Mbps (Media de bajada)": "Tot_Vel_Media_DL"},
                                    "Totales Accesos Por Tecnología": { "ADSL": "Tot_Acc_ADSL",
                                                                        "Cablemodem": "Tot_Acc_CaModem",
                                                                        "Fibra óptica": "Tot_Acc_FO",
                                                                        "Wireless": "Tot_Acc_Wireless",
                                                                        "Otros": "Tot_Acc_Otros"},
                                    "Penetracion-totales": {"Accesos por cada 100 hogares": "Tot_pntrcion_x_c100_Hoga",
                                                            "Accesos por cada 100 hab": "Tot_pntrcion_x_c100_Habi"},
                                    "Totales Accesos por rango": {"Hasta 512 kbps": "Tot_Acc_rango_0_512_kbps",
                                                                  "Entre 512 Kbps y 1 Mbps": "Tot_Acc_rango_0.512->1_Mbps",
                                                                  "Entre 1 Mbps y 6 Mbps": "Tot_Acc_rango_1->6_Mbps",
                                                                  "Entre 6 Mbps y 10 Mbps": "Tot_Acc_rango_6->10_Mbps",
                                                                  "Entre 10 Mbps y 20 Mbps": "Tot_Acc_rango_10->20_Mbps",
                                                                  "Entre 20 Mbps y 30 Mbps": "Tot_Acc_rango_20->30_Mbps",
                                                                  "Más de 30 Mbps": "Tot_Acc_rango_>30_Mbps",
                                                                  "OTROS": "Tot_Acc_rango_Otros",
                                                                  "Total": "Tot_Acc_rango"},
                                    "Totales Dial-BAf":{"Banda ancha fija": "Tot_B_Ancha_Fija",
                                                        "Dial up": "Tot_DialUp",
                                                        "Total": "Tot_DialUp_+_B_Ancha_Fija"},
                                    "Ingresos ": {"Ingresos": "Total_Ingresos"}
                                }

# Lista de nombres actuales y sus renombramientos para los campos de nivel nacional:
nuevos_nombres_nivel_provincias = { "Velocidad % por prov": {"Mbps (Media de bajada)": "Tot_Vel_Media_DL_Trim_x_Prov"},
                                "Accesos Por Tecnología": {"ADSL": "Tot_Acc_ADSL_x_Prov",
                                                           "Cablemodem": "Tot_Acc_CaModem_x_Prov",
                                                           "Fibra óptica": "Tot_Acc_FO_x_Prov",
                                                           "Wireless": "Tot_Acc_Wireless_x_Prov",
                                                           "Otros": "Tot_Acc_Otros_x_Prov",
                                                           "Total": "Tot_Acc_x_Prov"},
                                "Penetración-poblacion": {"Accesos por cada 100 hab": "Tot_pntrcion_x_c100_Habi_x_Prov"},
                                "Penetracion-hogares": {"Accesos por cada 100 hogares": "Tot_pntrcion_x_c100_Hoga_x_Prov"},
                                "Accesos por rangos": {"HASTA 512 kbps": "Tot_Acc_rango_0->512_kbps_x_Prov",
                                                       "+ 512 Kbps - 1 Mbps": "Tot_Acc_rango_0.512->1_Mbps_x_Prov",
                                                       "+ 1 Mbps - 6 Mbps": "Tot_Acc_rango_1->6_Mbps_x_Prov",
                                                       "+ 6 Mbps - 10 Mbps": "Tot_Acc_rango_6->10_Mbps_x_Prov",
                                                       "+ 10 Mbps - 20 Mbps": "Tot_Acc_rango_10->20_Mbps_x_Prov",
                                                       "+ 20 Mbps - 30 Mbps": "Tot_Acc_rango_20->30_Mbps_x_Prov",
                                                       "+ 30 Mbps": "Tot_Acc_rango_>30_Mbps_x_Prov",
                                                       "OTROS": "Tot_Acc_rango_Otros_Mbps_x_Prov",
                                                       "Total": "Tot_Acc_rango_Mbps_x_Prov"},
                                "Dial-BAf": {"Banda ancha fija": "Tot_B_Ancha_Fija_x_Prov",
                                             "Dial up": "Tot_DialUp_x_Prov",
                                             "Total": "Tot_DialUp_+_B_Ancha_Fija_x_Prov"}
                                }
# Renombramos los campos de las hojas con data de nivel Nacional.
df_internet = renombrar_campos(df_internet, nuevos_nombres_nivel_nacional)
# SRenombramos los campos de las hojas con data de nivel Provincia.
df_internet = renombrar_campos(df_internet, nuevos_nombres_nivel_provincias)

#**4. Para cada hoja validamos dimensionalidad e identificamos si se generaria o no perdida de 
# información evaluando el impacto que generaría unificar las hojas con los nuevos campos id.**

# Seleccionamos las hojas que contienen los campos Año, Trimestre y Provincia
hojas_a_excluir = []
campos_ano_trim_prov = ["Año", "Trimestre","Provincia"]
hojas_ano_trim_prov = obtener_hojas_validas(campos_ano_trim_prov, df_internet, hojas_a_excluir)
print("Se encontraron {} hojas que contienen los campos Año, Trimestre y Provincia:\n".format(len(hojas_ano_trim_prov)), hojas_ano_trim_prov, "\n")

campos_ano_trim = ["Año", "Trimestre"]
hojas_ano_trim = obtener_hojas_validas(campos_ano_trim, df_internet, hojas_ano_trim_prov)
print("Se encontraron {} hojas que contienen los campos Año y Trimestre solamente:\n".format(len(hojas_ano_trim)), hojas_ano_trim, "\n")

# Extraemos la lista de hojas que contienen los campos Año, Trimestre y Provincia de la tupla obtenida anteriormente
lista_hojas_ano_trim_prov = [nombre for nombre, _ in hojas_ano_trim_prov]
# Se descarta la hoja 'Velocidad_sin_Rangos' por tener una dimension de (18884, 5) a diferencia de las otras hojas que compraten dimensiones similares
lista_hojas_ano_trim_prov = ['Velocidad % por prov', 'Accesos Por Tecnología', 'Penetración-poblacion', 'Penetracion-hogares', 'Accesos por rangos', 'Dial-BAf']
print(lista_hojas_ano_trim_prov)

# Extraemos la lista de hojas que contienen los campos Año y Trimestre solamente de la tupla obtenida anteriormente
lista_hojas_ano_trim = [nombre for nombre, _ in hojas_ano_trim]
print(lista_hojas_ano_trim)

for hojas in df_internet:
    print(f'La hoja {hojas} tiene un tamaño de {df_internet[hojas].shape} y contiene los campos:')
    print(validar_df(df_internet[hojas]))
    print('\n')

La hoja Acc_vel_loc_sinrangos contiene:
        Partido   Localidad  link Indec Velocidad (Mbps)  Provincia  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.00      1.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.50      2.0
2  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.75     19.0


La hoja Velocidad_sin_Rangos contiene:
    Año  Trimestre     Provincia  Velocidad  Accesos
0  2024          2  BUENOS AIRES       75.0     1062
1  2024          2  BUENOS AIRES       59.0       59
2  2024          2  BUENOS AIRES      480.0        5


La hoja Accesos_tecnologia_localidad contiene:
      Provincia     Partido   Localidad    Tecnologia Link Indec  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          ADSL    6854100    755.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo    CABLEMODEM    6854100   4600.0
2  BUENOS AIRES  25 de Mayo  25 de Mayo  FIBRA OPTICA    6854100      2.0


La hoja Velocidad % por prov contiene:
    Año  Trim

In [13]:
print(validar_df(df_internet['Accesos por rangos']))

Advertencia: La columna 'Tot_Acc_rango_->_Mbps_x_Prov' aparece duplicada. Se usará la primera aparición.
Advertencia: La columna 'Tot_Acc_rango_->_Mbps_x_Prov' aparece duplicada. Se usará la primera aparición.
Advertencia: La columna 'Tot_Acc_rango_->_Mbps_x_Prov' aparece duplicada. Se usará la primera aparición.
Advertencia: La columna 'Tot_Acc_rango_->_Mbps_x_Prov' aparece duplicada. Se usará la primera aparición.
                                   Tipo de Dato   Int Float Bool DateT   Str Ctgory No_Nulos Nulos Únicos Ceros Vacíos (string)        Media      Desvi_Std   Mínimo    Q1_25%    Q2_50%     Q3_75%      Máximo Negativos
Año                                       int64  1000     0    0     0     0    NaN     1000     0     11     0             NaN     2018.824       3.058493     2014    2016.0    2019.0     2021.0        2024         0
Trimestre                                 int64  1000     0    0     0     0    NaN     1000     0      4     0             NaN        2.464    

In [3]:
from functions import validar_df
"""
print(lista_hojas_ano_trim)
print(validar_df(df_internet["Totales VMD"]))
print(validar_df(df_internet["Totales Accesos Por Tecnología"]))
print(validar_df(df_internet["Penetracion-totales"]))
print(validar_df(df_internet["Totales Accesos por rango"]))
print(validar_df(df_internet["Totales Dial-BAf"]))
print(validar_df(df_internet["Ingresos "]))
print(validar_df(df_internet["Accesos Por Tecnología"]))
print(validar_df(df_internet["Accesos por rangos"]))
"""
for hojas in df_internet:
    print(f'La hoja {hojas} tiene un tamaño de {df_internet[hojas].shape} y contiene los campos:')
    print(validar_df(df_internet[hojas]))
    print('\n')



La hoja Acc_vel_loc_sinrangos tiene un tamaño de (18864, 6) y contiene los campos:
                 Tipo de Dato    Int  Float Bool DateT    Str Ctgory Val_No_Nulos Val_Nulos Val_Únicos Val_Cero Val_Vacíos (string)       Media Desviación_Std  Mínimo Q1_25% Q2_50% Q3_75%    Máximo Negativos
Partido                object      0      0    0     0  18864    NaN        18864         0         24      NaN                   0         NaN            NaN     NaN    NaN    NaN    NaN       NaN       NaN
Localidad              object      0      0    0     0  18864    NaN        18864         0        435      NaN                   0         NaN            NaN     NaN    NaN    NaN    NaN       NaN       NaN
link Indec             object      0      0    0     0  18863    NaN        18863         1       2782      NaN                 NaN         NaN            NaN     NaN    NaN    NaN    NaN       NaN       NaN
Velocidad (Mbps)       object  18726      0    0     0    138    NaN        18864    

In [3]:
#**Aplicamos reduccionalidad al dataset luego de identificar la presencia de campos redundantes
#como "Trimestre.1" y campos comunes (Provincia, año, trimestre), creamos un campo id_año_trim_nal y
#id_año_trim_prov para todas las hojas donde aplique respectivamente:

def imprimir_info_anio_trimestre(df_dict):
    """
    Recorre cada DataFrame en el diccionario y muestra información detallada
    (head y describe) de las columnas "Año" y "Trimestre", si existen en cada hoja.
    
    Parámetros:
      - df_dict (dict): Diccionario donde la clave es el nombre de la hoja y el valor es el DataFrame.
    """
    for hoja, df in df_dict.items():
        print(f'La hoja {hoja} contiene:')
        # Selecciona las columnas de interés que existan en el DataFrame.
        columnas_interes = [col for col in ["Año", "Trimestre"] if col in df.columns]
        if columnas_interes:
            print("Primeras 3 filas:")
            print(df[columnas_interes].head(3))
            print("\nResumen estadístico:")
            print(df[columnas_interes].describe())
        else:
            print("No contiene los campos 'Año' y 'Trimestre'.")
        print("\n" + "-"*50 + "\n")

# Ejemplo de uso:
imprimir_info_anio_trimestre(df_internet)



La hoja Acc_vel_loc_sinrangos contiene:
No contiene los campos 'Año' y 'Trimestre'.

--------------------------------------------------

La hoja Velocidad_sin_Rangos contiene:
Primeras 3 filas:
    Año  Trimestre
0  2024          2
1  2024          2
2  2024          2

Resumen estadístico:
                Año     Trimestre
count  18884.000000  18884.000000
mean    2021.017104      2.477653
std        1.919740      1.126947
min     2017.000000      1.000000
25%     2019.000000      1.000000
50%     2021.000000      2.000000
75%     2023.000000      4.000000
max     2024.000000      4.000000

--------------------------------------------------

La hoja Accesos_tecnologia_localidad contiene:
No contiene los campos 'Año' y 'Trimestre'.

--------------------------------------------------

La hoja Velocidad % por prov contiene:
Primeras 3 filas:
    Año  Trimestre
0  2024          2
1  2024          2
2  2024          2

Resumen estadístico:
               Año    Trimestre
count  1008.000000

In [15]:
# Crear el DataFrame base con todas las combinaciones únicas de Año, Trimestre y Provincia.
# Puedes obtenerlo a partir de una de las hojas que sepas tiene esos campos o
# combinando los valores únicos de todas las hojas.
import pandas as pd
import itertools

# 1. Extraer valores únicos de "Año", "Trimestre" y "Provincia" de todas las hojas
valores_ano = set()
valores_trim = set()
valores_prov = set()

for sheet, df in df_internet.items():
    if "Año" in df.columns:
        # Convertimos a int para asegurar un tipo uniforme y omitimos NaN
        valores_ano.update([int(x) for x in df["Año"].dropna().unique()])
    if "Trimestre" in df.columns:
        # Convertimos a int para asegurar que se ordenen numéricamente
        valores_trim.update([int(x) for x in df["Trimestre"].dropna().unique()])
    if "Provincia" in df.columns:
        # Convertimos a str para evitar conflictos de tipo
        valores_prov.update([str(x) for x in df["Provincia"].dropna().unique()])

# 2. Convertir a listas y ordenarlas
valores_ano = sorted(valores_ano)
valores_trim = sorted(valores_trim)
valores_prov = sorted(valores_prov)


ValueError: invalid literal for int() with base 10: '2019 *'

In [ ]:

# Se identifica varias hojas tienen en comun los campos "Año" y "Trimestre", y otras hojas "Año", "Trimestre" y "Provincia"
# Se procede a crear un campo "id_año_trimestre" en cada hoja que contiene esos campos en comun y la ubicamos como primer campo
# Recorremos cada hoja y verificamos si contiene las columnas 'Año' y 'Trimestre'
from functions import opciones_impresion
opciones_impresion()
"""
for hoja, df in df_internet.items():
    if 'Año' in df.columns and 'Trimestre' in df.columns:
        df['id_Trimestre_Año'] = df['Trimestre'].astype(str) + '_' + df['Año'].astype(str)
        cols = df.columns.tolist()
        cols.remove('id_Trimestre_Año')
        df = df[['id_Trimestre_Año'] + cols]
        df_internet[hoja] = df

print(df_internet['Totales Accesos por rango'].head(5))
"""

  id_Trimestre_Año   Año  Trimestre  Tot_Acc_rango_0_512_kbps  Tot_Acc_rango_0.512->1_Mbps  Tot_Acc_rango_1->6_Mbps  Tot_Acc_rango_6->10_Mbps  Tot_Acc_rango_10->20_Mbps  Tot_Acc_rango_20->30_Mbps  Tot_Acc_rango_>30_Mbps  Tot_Acc_rango_Otros  Tot_Acc_rango
0           2_2024  2024          2                     28151                        67024                   840200                    911374                     662649                     348253                 8357088               341368       11556107
1           1_2024  2024          1                     28801                        69355                   866152                    950930                     672155                     353896                 8363694               328173       11633156
2           4_2023  2023          4                     29708                        71742                   900253                    978108                     697232                     350290                 8224736             

In [3]:
# Validamos las primeras lineas de cada hoja para identifica su contenido
for hojas in df_internet:
    print(f'La hoja {hojas} contiene:') # Imprime el nombre de la hoja
    print(df_internet[hojas].head(3))
    print('\n')

print(hojas_ano_trim_prov)

La hoja Acc_vel_loc_sinrangos contiene:
        Partido   Localidad  link Indec Velocidad (Mbps)  Provincia  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.00      1.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.50      2.0
2  BUENOS AIRES  25 de Mayo  25 de Mayo          6854100       0.75     19.0


La hoja Velocidad_sin_Rangos contiene:
    Año  Trimestre     Provincia  Velocidad  Accesos
0  2024          2  BUENOS AIRES       75.0     1062
1  2024          2  BUENOS AIRES       59.0       59
2  2024          2  BUENOS AIRES      480.0        5


La hoja Accesos_tecnologia_localidad contiene:
      Provincia     Partido   Localidad    Tecnologia Link Indec  Accesos
0  BUENOS AIRES  25 de Mayo  25 de Mayo          ADSL    6854100    755.0
1  BUENOS AIRES  25 de Mayo  25 de Mayo    CABLEMODEM    6854100   4600.0
2  BUENOS AIRES  25 de Mayo  25 de Mayo  FIBRA OPTICA    6854100      2.0


La hoja Velocidad % por prov contiene:
    Año  Trim

In [4]:
from functions import fusionar_por_campos, opciones_impresion
opciones_impresion()

df_internet_provincia = fusionar_por_campos(["Año","Trimestre","Provincia"],lista_hojas_ano_trim_prov,df_internet)
print(df_internet_provincia.head(10))


    Año Trimestre        Provincia  Tot_Vel_Media_DL_Trim_x_Prov  Tot_Acc_ADSL_x_Prov  Tot_Acc_CaModem_x_Prov  Tot_Acc_FO_x_Prov  Tot_Acc_Wireless_x_Prov  Tot_Acc_Otros_x_Prov  Tot_Acc_x_Prov  Tot_pntrcion_x_c100_Habi_x_Prov  Tot_pntrcion_x_c100_Hoga_x_Prov  Tot_Acc_rango_0_512_kbps_x_Prov  Tot_Acc_rango_0.512->1_Mbps_x_Prov  Tot_Acc_rango_1->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_Otros_Mbps_x_Prov  Tot_Acc_rango_Mbps_x_Prov  Tot_B_Ancha_Fija_x_Prov  Tot_DialUp_x_Prov  Tot_DialUp_+_B_Ancha_Fija_x_Prov
0  2014         1     Buenos Aires                      3.733133            1567685.0               1000879.0           120960.0                  16528.0               33824.0       2739876.0                        16.692346                        54.570073                           8469.0                           171244.48                      2279875.0                     121787

In [9]:
# Para verificar duplicados en las columnas "Año", "Trimestre" y "Provincia" en un DataFrame:
duplicados = df_internet["Penetracion-hogares"][df_internet["Penetracion-hogares"].duplicated(subset=["Año", "Trimestre", "Provincia"], keep=False)]
print(duplicados)

Empty DataFrame
Columns: [Año, Trimestre, Provincia, Tot_pntrcion_x_c100_Hoga_x_Prov]
Index: []


In [16]:
from functions import opciones_impresion
print(df_internet_provincia.head(10))

  id_Trimestre_Año   Año Trimestre        Provincia  Tot_Vel_Media_DL_Trim_x_Prov id_Trimestre_Año_dup  Tot_Acc_ADSL_x_Prov  Tot_Acc_CaModem_x_Prov  Tot_Acc_FO_x_Prov  Tot_Acc_Wireless_x_Prov  Tot_Acc_Otros_x_Prov  Tot_Acc_x_Prov id_Trimestre_Año_dup  Tot_pntrcion_x_c100_Habi_x_Prov id_Trimestre_Año_dup  Tot_pntrcion_x_c100_Hoga_x_Prov id_Trimestre_Año_dup  Tot_Acc_rango_0_512_kbps_x_Prov  Tot_Acc_rango_0.512->1_Mbps_x_Prov  Tot_Acc_rango_1->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_->_Mbps_x_Prov  Tot_Acc_rango_Otros_Mbps_x_Prov  Tot_Acc_rango_Mbps_x_Prov i

In [ ]:
# Modifico el nombre de los campos para hacerlos mas intuitivos
#df_internet['Internet'].rename(columns={'Año':'año', 'Trimestre':'trimestre', 'Provincia':'provincia', 'Hogares con acceso a Internet':'hogares

In [34]:
import pandas as pd
from functions import obtener_hojas_validas

campos_ano_trim_prov = ["Año", "Trimestre","Provincia"]
hojas_a_excluir = []

hojas_ano_trim_prov = obtener_hojas_validas(campos_ano_trim_prov, df_internet, hojas_a_excluir)
print("Hojas que contienen los campos año, trimestre y provincia:\n", hojas_ano_trim_prov)
print("\nTotal de hojas encontradas:", len(hojas_ano_trim_prov))

campos_ano_trim = ["Año", "Trimestre"]
hojas_ano_trim = obtener_hojas_validas(campos_ano_trim, df_internet, hojas_ano_trim_prov)
print("Hojas que contienen los campos año y trimestre solamente:\n", hojas_ano_trim)
print("\nTotal de hojas encontradas:", len(hojas_ano_trim))

ImportError: cannot import name 'obtener_hojas_validas' from 'functions' (/Users/usuario/Documents/M7_LABs_PI/mvp_pi2/functions.py)

In [ ]:
import pandas as pd

def fusionar_por_campo(campo_id, lista_hojas, df_dict):
    """
    Fusiona las hojas indicadas en 'lista_hojas' usando el campo único 'campo_id' como llave de unión.
    
    Parámetros:
      - campo_id (str): Nombre del campo único que se utilizará para fusionar (ej. "id_año_trimestre").
      - lista_hojas (list): Lista de nombres de las hojas a fusionar.
      - df_dict (dict): Diccionario con los DataFrames (clave: nombre de la hoja, valor: DataFrame).
      
    Retorna:
      DataFrame: Resultado de la fusión externa de los DataFrames indicados.
    """
    df_fusionado = None
    
    for hoja in lista_hojas:
        if hoja not in df_dict:
            print(f"La hoja '{hoja}' no se encuentra en el diccionario. Se omite.")
            continue
        df = df_dict[hoja]
        if campo_id not in df.columns:
            print(f"La hoja '{hoja}' no contiene el campo '{campo_id}'. Se omite.")
            continue
        
        if df_fusionado is None:
            df_fusionado = df.copy()
        else:
            df_fusionado = pd.merge(df_fusionado, df, on=campo_id, how='outer', suffixes=('', '_dup'))
    
    if df_fusionado is None:
        return pd.DataFrame()
    return df_fusionado

# Ejemplo de uso:
# Supongamos que 'df_internet' es el diccionario obtenido con pd.read_excel(..., sheet_name=None)
# Fusionar hojas usando el campo "id_año_trimestre"
df_resultado = fusionar_por_campo("id_año_trimestre", ["Velocidad_sin_Rangos", "Totales VMD", "Ingresos"], df_internet)

In [ ]:
"""
import pandas as pd

def unificar_hojas(df_dict):
    
    Unifica hojas de un archivo Excel en dos DataFrames:
    1. "comunes_ano_trimestre": Hojas que tienen los campos "Año" y "Trimestre".
    2. "comunes_ano_trimestre_provincia": Hojas que tienen los campos "Año", "Trimestre" y "Provincia".
    
    La función también genera y reordena el identificador (id_año_trimestre o id_año_trimestre_prov)
    para que quede en la primera posición de cada DataFrame.
    
    Parámetros:
        df_dict (dict): Diccionario donde la llave es el nombre de la hoja y el valor es el DataFrame.
        
    Retorna:
        dict: Diccionario con dos nuevos DataFrames:
            - "comunes_ano_trimestre"
            - "comunes_ano_trimestre_provincia"
    
    dfs_ano_trimestre = []
    dfs_ano_trimestre_provincia = []
    
    for hoja, df in df_dict.items():
        # Hojas que contienen "Año" y "Trimestre"
        if all(col in df.columns for col in ['Año', 'Trimestre']):
            # Creamos el identificador si no existe
            if 'id_año_trimestre' not in df.columns:
                df['id_año_trimestre'] = df['Año'].astype(str) + '-' + df['Trimestre'].astype(str)
            # Reordenamos para ubicar "id_año_trimestre" en la primera posición
            cols = df.columns.tolist()
            cols.remove('id_año_trimestre')
            df = df[['id_año_trimestre'] + cols]
            dfs_ano_trimestre.append(df)
        
        # Hojas que contienen "Año", "Trimestre" y "Provincia"
        if all(col in df.columns for col in ['Año', 'Trimestre', 'Provincia']):
            if 'id_año_trimestre_prov' not in df.columns:
                df['id_año_trimestre_prov'] = (
                    df['Año'].astype(str) + '-' + 
                    df['Trimestre'].astype(str) + '-' + 
                    df['Provincia'].astype(str).str.upper()
                )
            # Reordenamos para ubicar "id_año_trimestre_prov" en la primera posición
            cols = df.columns.tolist()
            cols.remove('id_año_trimestre_prov')
            df = df[['id_año_trimestre_prov'] + cols]
            dfs_ano_trimestre_provincia.append(df)
    
    # Unificamos (concatenamos) los DataFrames encontrados
    comunes_ano_trimestre = pd.concat(dfs_ano_trimestre, ignore_index=True, sort=False) if dfs_ano_trimestre else pd.DataFrame()
    comunes_ano_trimestre_provincia = pd.concat(dfs_ano_trimestre_provincia, ignore_index=True, sort=False) if dfs_ano_trimestre_provincia else pd.DataFrame()
    
    return {
         "comunes_ano_trimestre": comunes_ano_trimestre,
         "comunes_ano_trimestre_provincia": comunes_ano_trimestre_provincia
    }

# Ejemplo de uso:
# Suponiendo que ya tienes el diccionario df_internet con cada hoja cargada:
unificados = unificar_hojas(df_internet)
df_comunes = unificados["comunes_ano_trimestre"]
df_comunes_prov = unificados["comunes_ano_trimestre_provincia"]
df_comunes.info()
"""